# Graphene/GaN

In [1]:
import sys
sys.path.append('../interfacemaster/')

In [1]:
from interfacemaster.cellcalc import get_primitive_hkl, get_pri_vec_inplane, get_normal_index, get_normal_from_MI, rot
from interfacemaster.interface_generator import core, convert_vector_index
from interfacemaster.symmetric_tilt import get_csl_twisted_graphenes
from numpy import array, dot, round, cross, eye, pi, ceil, column_stack, arange
from numpy.linalg import inv, det, norm
from gb_code.csl_generator import print_list
import pandas as pd

In [18]:
from pymatgen.io.vasp import Poscar
from pymatgen.io.cif import CifWriter

p = Poscar.from_file('POSCAR_two')
w = CifWriter(p.structure)
w.write_file('GaN_graphene.cif')

In [19]:
my_interface = core('GaN_graphene.cif', 'cif_files/Si_mp-149_conventional_standard.cif')

Warning!, this programme will rewrite the POSCAR file in this dir!


In [13]:
my_interface = core('cif_files/C_mp-990448_conventional_standard.cif',\
                    'cif_files/GaN_mp-804_conventional_standard.cif')

Warning!, this programme will rewrite the POSCAR file in this dir!


## Searching two_dimensional CSL

In [20]:
#combining two specified surfaces
hkl_1 = get_primitive_hkl([1, 0, 0], my_interface.conv_lattice_1, my_interface.lattice_1)
hkl_2 = get_primitive_hkl([1, 1, 0], my_interface.conv_lattice_2, my_interface.lattice_2)

In [21]:
my_interface.parse_limit(du = 1e-1, S  = 1e-1, sgm1=50, sgm2=50, dd = 1e-1)
my_interface.search_one_position_2D(hkl_1, hkl_2, theta_range = 30, \
                                    dtheta = 0.01, integer_tol = 1e-4, start = 0)

Congrates, we found an appx CSL!

U1 = 
[[ 0  7]
 [ 0  0]
 [-1  0]]; sigma_1 = 7

U2 = 
[[ 3  0]
 [-3  9]
 [-3 -9]]; sigma_2 = 66

D = 
[[ 1.         -0.         -0.        ]
 [ 0.          0.99296184  0.        ]
 [ 0.          0.          1.04239279]]

axis = [8.44281605e+01 5.16973382e-15 0.00000000e+00] ; theta = 0.0



## Get bicrystal

In [22]:
hkl = get_primitive_hkl([0, 0, 1], my_interface.conv_lattice_1, my_interface.lattice_1)
my_interface.compute_bicrystal_two_D(hkl_1, hkl_2, normal_ortho = False, lim = 50, tol_ortho = 1e-2, tol_integer=1e-3)

cell 1:
[[ 0  0  7]
 [-1  0  0]
 [ 0 -1  0]]
cell 2:
[[ 1  3  0]
 [ 0 -3  9]
 [ 0 -3 -9]]


In [25]:
help(my_interface.get_bicrystal)

Help on method get_bicrystal in module interfacemaster.interface_generator:

get_bicrystal(dydz=array([0., 0., 0.]), dx=0, dp1=0, dp2=0, xyz_1=[1, 1, 1], xyz_2=[1, 1, 1], vx=0, filename='POSCAR', two_D=False, filetype='VASP', mirror=False, KTI=False) method of interfacemaster.interface_generator.core instance
    generate a cif file for the bicrystal structure
    
    Parameters
    ----------
    dydz : numpy array
        translation vector in the interface
    dx : float
        translation normal to the interface
    dp1, dp2 : float
        termination of slab 1, 2
    xyz1, xyz2 : list
        expansion of slab 1, 2
    vx : float
        vacuum spacing, default 0
    filename : str
        filename, default 'POSCAR'
    two_D : bool
        whether a two CSL
    filetype : str
        filetype, 'VASP' or 'LAMMPS', default 'VASP'
    mirror : bool
        mirror, default False
    KTI : bool
        KTI, default False



In [31]:
layer_displacement = 3.4
my_interface.get_bicrystal(xyz_1 = [1,1,1], xyz_2 = [1,1,1], \
                           filetype= 'VASP', dydz = [0, 0, -2], dp1 = -0.5)

## Sample CNID 

In [30]:
gridsize = 0.3
n1 = int(ceil(norm(my_interface.CNID[:,0])/gridsize))
n2 = int(ceil(norm(my_interface.CNID[:,1])/gridsize))

In [38]:
import shutil
try:
    shutil.rmtree('CNID_inputs')
except:
    print('No existing files')
my_interface.sample_CNID([n1,n2], dp1 = 9.99905800e+00 - layer_displacement)

CNID
[[ 0.42857143 -0.28571429]
 [ 0.14285714 -0.42857143]
 [ 0.          0.        ]]
making 16 files...
completed


Please check the POSCAR file